<a href="https://colab.research.google.com/github/ejazfahil/Hardware-and-software-project/blob/main/FSF_StreamLit_HW_SW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit
!pip install pmdarima


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.4 MB/s eta 0:00:00


In [2]:
'''
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
'''

'\n!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip\n!unzip ngrok-stable-linux-amd64.zip\n'

In [3]:
# get_ipython().system_raw('./ngrok http 8501 &')

In [4]:
'''
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

'''

'\n!curl -s http://localhost:4040/api/tunnels | python3 -c     "import sys, json; print(json.load(sys.stdin)[\'tunnels\'][0][\'public_url\'])"\n\n'

In [5]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

SecretNotFoundError: Secret KAGGLE_KEY does not exist.

In [6]:
!kaggle datasets download -d anlgrbz/student-demographics-online-education-dataoulad

! unzip "student-demographics-online-education-dataoulad.zip"

Dataset URL: https://www.kaggle.com/datasets/anlgrbz/student-demographics-online-education-dataoulad
License(s): Attribution 4.0 International (CC BY 4.0)
100% 42.0M/42.2M [00:00<00:00, 46.3MB/s]
100% 42.2M/42.2M [00:00<00:00, 50.1MB/s]
Archive:  student-demographics-online-education-dataoulad.zip
  inflating: assessments.csv         
  inflating: courses.csv             
  inflating: studentAssessment.csv   
  inflating: studentInfo.csv         
  inflating: studentRegistration.csv  
  inflating: studentVle.csv          
  inflating: vle.csv                 


In [7]:
%%writefile app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error , mean_absolute_error, r2_score
from pmdarima.arima.utils import nsdiffs
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from xgboost import XGBRegressor



st.set_page_config(
    page_title='Open University Learning Analytics Dataset (OULAD) Analysis',
    page_icon='📈'
)

# -------------------------------------Functions--------------------------------------------
# -------------------------------------Functions--------------------------------------------
# -------------------------------------Functions--------------------------------------------
# -------------------------------------Functions--------------------------------------------


@st.cache_data
def load_data():
    assesment = pd.read_csv('/content/assessments.csv')
    course = pd.read_csv('/content/courses.csv')
    as_stu = pd.read_csv('/content/studentAssessment.csv')
    info_stu = pd.read_csv('/content/studentInfo.csv')
    reg_stu = pd.read_csv('/content/studentRegistration.csv')
    vle_stu = pd.read_csv('/content/studentVle.csv')
    vle = pd.read_csv('/content/vle.csv')
    return assesment, course, as_stu, info_stu, reg_stu, vle_stu, vle

assesment, course, as_stu, info_stu, reg_stu, vle_stu, vle = load_data()

def missingValueAssessment(data):
    st.write("## Data Information")
    buffer = io.StringIO()
    data.info(buf=buffer)
    s = buffer.getvalue()
    st.text(s)

    st.write("Dataframe Shape:: ", data.shape)

    st.write("----------------------------------------------------------------------------------")

    missing_values = data.isnull().sum()
    st.write("### Missing Values")
    st.write(missing_values)

    st.write("----------------------------------------------------------------------------------")

    missing_values_percentage = (missing_values / len(data)) * 100

    # Create a DataFrame to display missing values
    missing_values_df = pd.DataFrame({
        'Column': missing_values.index,
        'Missing Values': missing_values.values,
        'Percentage': missing_values_percentage
    })

    # Display the missing values
    st.write("### Missing Values DataFrame")
    st.write(missing_values_df)

    st.write("----------------------------------------------------------------------------------")

    # Visualize the missing values
    st.write("### Missing Values Visualization")
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Percentage', y='Column', data=missing_values_df.sort_values(by='Percentage', ascending=False))
    plt.title('Percentage of Missing Values by Column')
    plt.xlabel('Percentage of Missing Values')
    plt.ylabel('Columns')

    st.pyplot(plt)


def plotActivityCounts(data):
    activity_counts = data['activity_type'].value_counts().reset_index()
    activity_counts.columns = ['activity_type', 'count']

    # Sort the activity types by count in descending order
    activity_counts = activity_counts.sort_values(by='count', ascending=False)

    # Plot the bar chart
    plt.figure(figsize=(14, 7))
    sns.set_style("whitegrid")
    bar_plot = sns.barplot(x='activity_type', y='count', data=activity_counts, palette='husl', hue='activity_type', dodge=False)

    # Set the title and labels
    plt.title('Count of Each Activity Type', fontsize=16)
    plt.xlabel('Activity Type', fontsize=14)
    plt.ylabel('Count', fontsize=14)

    # Rotate the x labels for better readability
    bar_plot.set_xticklabels(bar_plot.get_xticklabels(), rotation=45, horizontalalignment='right')

    st.pyplot(plt.gcf())  # Use st.pyplot to display the plot in Streamlit

def student_vle_eda(data):
  daily_interactions = data.groupby('date')['sum_click'].sum().reset_index()

  # Set date as index
  daily_interactions.set_index('date', inplace=True)

  # Display the first few rows
  print(daily_interactions.head())

  # Plot the time series data
  plt.figure(figsize=(14, 7))
  plt.plot(daily_interactions['sum_click'], label='Daily Interactions')
  plt.title('Daily Student Interactions with VLE')
  plt.xlabel('Date')
  plt.ylabel('Sum of Clicks')
  plt.legend()
  st.pyplot(plt.gcf())

def stu_reg_eda(data):

  st.subheader('Registration Count of Students With Time')
  #Aggregate interactions per day
  daily_interactions = data.groupby('date_registration')['id_student'].sum().reset_index()

  # Set date as index
  daily_interactions.set_index('date_registration', inplace=True)

  # Display the first few rows
  print(daily_interactions.head())

  # Plot the time series data
  plt.figure(figsize=(14, 7))
  plt.plot(daily_interactions['id_student'], label='Registration Count')
  plt.title('Count of Student Registration with time')
  plt.xlabel('Date of Registration')
  plt.ylabel('Count of Students')
  plt.legend()
  st.pyplot(plt.gcf())



  st.subheader('Registration Count Per Module of Students With Time')

  # Aggregate the registration counts per module over time
  registration_trends = data.groupby(['date_registration', 'code_module']).size().reset_index(name='count')

  # Get the list of unique modules
  modules = registration_trends['code_module'].unique()

  # Set the figure size for each plot
  plt.figure(figsize=(14, 7))

  # Loop through each module and create a separate plot
  for module in modules:
      plt.figure(figsize=(14, 7))
      module_data = registration_trends[registration_trends['code_module'] == module]
      sns.lineplot(x='date_registration', y='count', data=module_data, marker='o')

      # Set the title and labels
      plt.title(f'Student Registration Trends for Module {module}', fontsize=16)
      plt.xlabel('Date of Registration', fontsize=14)
      plt.ylabel('Count of Registrations', fontsize=14)

      # Rotate the x-axis labels for better readability
      plt.xticks(rotation=45)

      # Display the plot
      st.pyplot(plt.gcf())


def info_stu_eda(info_stu):

  st.subheader('Head of the data frame')

  st.write(info_stu.head(10))

  st.subheader('Count Of Exam Results')

  activity_counts = info_stu['final_result'].value_counts().reset_index()
  activity_counts.columns = ['final_result', 'count']

  # Sort the final_result by count in descending order
  activity_counts = activity_counts.sort_values(by='count', ascending=False)

  # Plot the bar chart
  plt.figure(figsize=(14, 7))
  sns.set_style("whitegrid")
  bar_plot = sns.barplot(x='final_result', y='count', data=activity_counts, palette='Set2', hue = "final_result", legend=False)

  # Set the title and labels
  plt.title('Count of Exam Result', fontsize=16)
  plt.xlabel('Result Status', fontsize=14)
  plt.ylabel('Count', fontsize=14)

  # Rotate the x labels for better readability
  bar_plot.set_xticklabels(bar_plot.get_xticklabels(), rotation=45, horizontalalignment='right')

  # Display the plot
  st.pyplot(plt.gcf())

  #--------------------------------------------------------------------------------------------------------------

  st.subheader('Final Results by Gender')

  # Aggregate data by final result and gender
  final_result_gender_counts = info_stu.groupby(['final_result', 'gender']).size().reset_index(name='count')

  # Sort final results by count in descending order
  sorted_results = final_result_gender_counts.groupby('final_result').sum().sort_values(by='count', ascending=False).index

  # Plot the bar chart
  plt.figure(figsize=(14, 7))
  sns.barplot(x='final_result', y='count', hue='gender', data=final_result_gender_counts, palette='Set2', order=sorted_results)

  # Set the title and labels
  plt.title('Final Results by Gender (Sorted)', fontsize=16)
  plt.xlabel('Final Result', fontsize=14)
  plt.ylabel('Count', fontsize=14)

  # Display the plot
  st.pyplot(plt.gcf())

  #--------------------------------------------------------------------------------------------------------------

  st.subheader('Final Results by Region')

  # Aggregate data by final result and region
  final_result_region_counts = info_stu.groupby(['final_result', 'region']).size().reset_index(name='count')

  # Sort final results by count in descending order
  sorted_results = final_result_region_counts.groupby('final_result').sum().sort_values(by='count', ascending=False).index

  # Pivot the data to have regions as columns
  pivot_data = final_result_region_counts.pivot(index='final_result', columns='region', values='count').reindex(sorted_results).fillna(0)

  # Plot the stacked bar chart
  pivot_data.plot(kind='bar', stacked=True, figsize=(14, 7), colormap='Set2')

  # Set the title and labels
  plt.title('Final Results by Region (Stacked)', fontsize=16)
  plt.xlabel('Final Result', fontsize=14)
  plt.ylabel('Count', fontsize=14)
  plt.xticks(rotation=45)

  # Display the plot
  st.pyplot(plt.gcf())

  #--------------------------------------------------------------------------------------------------------------

  st.subheader('Final Results by Highest Education')

  # Aggregate data by final result and highest education
  final_result_education_counts = info_stu.groupby(['final_result', 'highest_education']).size().reset_index(name='count')

  # Sort final results by count in descending order
  sorted_results = final_result_education_counts.groupby('final_result').sum().sort_values(by='count', ascending=False).index

  # Pivot the data to have highest education as columns
  pivot_data = final_result_education_counts.pivot(index='final_result', columns='highest_education', values='count').reindex(sorted_results).fillna(0)

  # Plot the stacked bar chart
  pivot_data.plot(kind='bar', stacked=True, figsize=(14, 7), colormap='Set2')

  # Set the title and labels
  plt.title('Final Results by Highest Education (Stacked)', fontsize=16)
  plt.xlabel('Final Result', fontsize=14)
  plt.ylabel('Count', fontsize=14)
  plt.xticks(rotation=45)

  # Display the plot
  st.pyplot(plt.gcf())

  #--------------------------------------------------------------------------------------------------------------

  st.subheader('Final Results by Module')

  # Aggregate data by final result and highest education
  final_result_education_counts = info_stu.groupby(['final_result', 'code_module']).size().reset_index(name='count')

  # Sort final results by count in descending order
  sorted_results = final_result_education_counts.groupby('final_result').sum().sort_values(by='count', ascending=False).index

  # Pivot the data to have highest education as columns
  pivot_data = final_result_education_counts.pivot(index='final_result', columns='code_module', values='count').reindex(sorted_results).fillna(0)

  # Plot the stacked bar chart
  pivot_data.plot(kind='bar', stacked=True, figsize=(14, 7), colormap='Set2')

  # Set the title and labels
  plt.title('Final Results by Module (Stacked)', fontsize=16)
  plt.xlabel('Final Result', fontsize=14)
  plt.ylabel('Count', fontsize=14)
  plt.xticks(rotation=45)

  # Display the plot
  st.pyplot(plt.gcf())

  #--------------------------------------------------------------------------------------------------------------

  st.subheader('Final Results by Module Grouped by Module')

  mod = info_stu['code_module'].unique()

  for mname in mod:
    data = info_stu[info_stu['code_module'] == mname]

    final_result_education_counts = data.groupby(['final_result', 'highest_education']).size().reset_index(name='count')

    # Sort final results by count in descending order
    sorted_results = final_result_education_counts.groupby('final_result').sum().index

    # Pivot the data to have highest education as columns
    pivot_data = final_result_education_counts.pivot(index='final_result', columns='highest_education', values='count').reindex(sorted_results).fillna(0)

    # Plot the stacked bar chart
    pivot_data.plot(kind='bar', stacked=True, figsize=(14, 7), colormap='Set2')

    # Set the title and labels
    plt.title(f'Final Results by Highest Education (Stacked) - Module {mname}', fontsize=16)
    plt.xlabel('Final Result', fontsize=14)
    plt.ylabel('Count', fontsize=14)
    plt.xticks(rotation=45)

    # Display the plot
    st.pyplot(plt.gcf())

  #--------------------------------------------------------------------------------------------------------------

  st.subheader('Correlation Matrix')

  student_info = info_stu.copy()

  # Preprocess the data
  # Convert categorical columns to numerical
  student_info['gender'] = student_info['gender'].map({'M': 0, 'F': 1})
  student_info['region'] = student_info['region'].astype('category').cat.codes
  student_info['highest_education'] = student_info['highest_education'].astype('category').cat.codes
  student_info['imd_band'] = student_info['imd_band'].astype('category').cat.codes
  student_info['age_band'] = student_info['age_band'].astype('category').cat.codes
  student_info['disability'] = student_info['disability'].map({'N': 0, 'Y': 1})
  student_info['code_module'] = student_info['code_module'].astype('category').cat.codes
  student_info['code_presentation'] = student_info['code_presentation'].astype('category').cat.codes

  # Map final result to numerical values
  final_result_map = {
      'Distinction': 4,
      'Pass': 3,
      'Fail': 2,
      'Withdrawn': 1
  }
  student_info['final_result'] = student_info['final_result'].map(final_result_map)

  # Calculate the correlation matrix
  correlation_matrix = student_info.corr()

  # Plot the correlation matrix
  plt.figure(figsize=(12, 8))
  sns.heatmap(correlation_matrix, annot=True, cmap='Set2', fmt='.2f', linewidths=0.5)

  # Set the title
  plt.title('Correlation Matrix with Final Result as Target', fontsize=16)

  # Display the plot
  st.pyplot(plt.gcf())

def as_stu_eda(as_stu):
  st.subheader('Head of Student Information Data')

  st.write(as_stu.head(10))

  st.subheader('Score Trend Over Time')
  Average_score = as_stu.groupby('date_submitted')['score'].mean().reset_index()
  Average_score['Score_Category'] = pd.cut(Average_score['score'], bins=[-float('inf'), 40, float('inf')], labels=['Fail (< 40)', 'Pass (>= 40)'])

  # Create a simple trend plot
  plt.figure(figsize=(10, 6))
  sns.lineplot(data=Average_score, x='date_submitted', y='score', marker='o', style='Score_Category', hue='Score_Category', palette='Set2', dashes = False)

  # Add title and labels
  plt.title('Score Trend Over Time')
  plt.xlabel('Date')
  plt.ylabel('Score')

  # Rotate the x labels for better readability
  plt.xticks(rotation=45)

  # Add grid
  plt.grid(True)

  # Show plot
  plt.tight_layout()
  st.pyplot(plt.gcf())

@st.cache_data
def data_merge(as_stu, info_stu):

  info_stu_cleaned = info_stu.copy()
  info_stu_cleaned['imd_band'] = info_stu_cleaned['imd_band'].fillna('unknown', inplace=True)


  as_stu_cleaned = as_stu.copy()
  student_means = as_stu_cleaned.groupby('id_student')['score'].transform('mean')
  as_stu_cleaned = as_stu_cleaned.dropna()
  #st.write(as_stu_cleaned.isna().sum())
  #as_stu_cleaned['score'] = as_stu_cleaned['score'].fillna(student_means[student_means['id_student'] == as_stu_cleaned['id_student']], inplace=True)
  as_stu_cleaned['date_submitted'] = as_stu_cleaned['date_submitted'].apply(lambda x: start_date + pd.Timedelta(days=x-1))

  data = pd.merge(as_stu_cleaned, info_stu_cleaned, on=['id_student'], how='left')
  data['score'] = data['score'].fillna(0)

  # Convert categorical columns to numerical
  data['gender'] = data['gender'].map({'M': 0, 'F': 1})
  data['region'] = data['region'].astype('category').cat.codes
  data['highest_education'] = data['highest_education'].astype('category').cat.codes
  data['imd_band'] = data['imd_band'].astype('category').cat.codes
  data['age_band'] = data['age_band'].astype('category').cat.codes
  data['disability'] = data['disability'].map({'N': 0, 'Y': 1})
  data['code_module'] = data['code_module'].astype('category').cat.codes
  data['code_presentation'] = data['code_presentation'].astype('category').cat.codes
  final_result_map = {
      'Distinction': 4,
      'Pass': 3,
      'Fail': 2,
      'Withdrawn': 1
  }
  data['final_result'] = data['final_result'].map(final_result_map)

  return data

def decision_tree(student_info_assessments):

    st.subheader('Decision Tree Regressor')

    st.write("""
    my_dt = DecisionTreeRegressor(
        max_depth=10,                 Limits depth of the tree
        min_samples_split=15,        Requires at least 15 samples to consider a split
        min_samples_leaf=5,         Requires at least 5 samples per leaf
        max_leaf_nodes=200,          Maximum number of leaf nodes
    )
    my_dt.fit(X_train, y_train)  # Fit to training data
    """)

    # Drop rows with any missing values in the features and target
    data = student_info_assessments.dropna(subset=['score'])
    X = data.drop(columns=['id_student', 'score']).select_dtypes(include=[np.number])
    y = data['score']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the DecisionTreeRegressor with specific parameters to regularize the tree
    my_dt = DecisionTreeRegressor(
        max_depth=10,                # Limits depth of the tree
        min_samples_split=15,       # Requires at least 20 samples to consider a split
        min_samples_leaf=5,        # Requires at least 10 samples per leaf
        max_leaf_nodes=200,         # Maximum number of leaf nodes
    )
    my_dt.fit(X_train, y_train)  # Fit to training data

    # Predict and calculate R2 score
    y_pred = my_dt.predict(X_test)

    st.write("MSE: ", mean_squared_error(y_test, y_pred))
    st.write("MAE: ", mean_absolute_error(y_test, y_pred))
    st.write("R2 Score: ", r2_score(y_test, y_pred))

    # Plot the decision tree
    plt.figure(figsize=(20, 10))  # Set the size of the plot according to your preference
    plot_tree(my_dt, feature_names=X.columns, filled=True)
    plt.title('Decision Tree Visualization')
    st.pyplot(plt.gcf())

    # Plot actual vs predicted values
    plt.figure(figsize=(14, 7))
    plt.plot(y_test.values, label='Actual Values', marker='o')
    plt.plot(y_pred, label='Predicted Values', marker='x')
    plt.title('Actual vs Predicted Values')
    plt.xlabel('Index')
    plt.ylabel('Score')
    plt.legend()
    st.pyplot(plt.gcf())

    st.subheader('Decision Tree Regressor with Pruning')

    # Get the cost complexity pruning path
    path = my_dt.cost_complexity_pruning_path(X_train, y_train)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities

    st.write('Total Impurity vs effective alpha for training set')

    plt.figure(figsize=(10, 6))
    plt.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
    plt.xlabel("effective alpha")
    plt.ylabel("total impurity of leaves")
    plt.title("Total Impurity vs effective alpha for training set")
    st.pyplot(plt.gcf())

    alpha_selected = ccp_alphas[2]

    st.write('Optimal alpha selected is:: ', alpha_selected)

    # Re-train the tree with the selected alpha
    my_dt_pruned = DecisionTreeRegressor(
        random_state=44,
        max_depth=10,
        min_samples_split=15,
        min_samples_leaf=5,
        max_leaf_nodes=20,
        ccp_alpha=alpha_selected
    )
    my_dt_pruned.fit(X_train, y_train)

    # Evaluate the pruned tree
    y_pred = my_dt_pruned.predict(X_test)
    st.write("MSE after pruning:", mean_squared_error(y_test, y_pred))
    st.write("MAE after pruning:", mean_absolute_error(y_test, y_pred))
    st.write("R2 Score: ", r2_score(list(y_test), list(my_dt_pruned.predict(X_test))))

    # Plot the decision tree
    plt.figure(figsize=(20,10))  # Set the size of the plot according to your preference
    plot_tree(my_dt_pruned, feature_names=X.columns, filled=True)
    plt.title('Pruned Decision Tree Visualization')
    st.pyplot(plt.gcf())

    # Plot actual vs predicted values
    plt.figure(figsize=(14, 7))
    plt.plot(y_test.values, label='Actual Values', marker='o')
    plt.plot(y_pred, label='Predicted Values', marker='x')
    plt.title('Actual vs Predicted Values')
    plt.xlabel('Index')
    plt.ylabel('Score')
    plt.legend()
    st.pyplot(plt.gcf())


def linear_reg(student_info_assessments):

  st.subheader('Linear Regression')

  st.write("""
  my_lm = LinearRegression()
  """)


  data = student_info_assessments.dropna(subset=['score'])
  X = data.drop(columns=['id_student', 'score']).select_dtypes(include=[np.number])
  y = data['score']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  my_lm = LinearRegression()
  my_lm.fit(X = X_train, y = y_train)

  y_pred = my_lm.predict(X_test)
  st.write("MSE ", mean_squared_error(y_test, y_pred))
  st.write("MAE ", mean_absolute_error(y_test, y_pred))
  st.write("R2_score ", r2_score(y_test, y_pred))

  results = pd.DataFrame({'Actual': y_test[1:1000], 'Predicted': y_pred[1:1000]})

  # Plot the results
  plt.figure(figsize=(10, 6))
  sns.scatterplot(x='Actual', y='Predicted', data=results, alpha=0.5, color='green')
  sns.lineplot(x='Actual', y='Actual', data=results, color='red')  # Diagonal line

  # Add titles and labels
  plt.title('Actual vs Predicted Scores')
  plt.xlabel('Actual Scores')
  plt.ylabel('Predicted Scores')
  st.pyplot(plt.gcf())


def xgboost_mdl(student_info_assessments):

  st.subheader('XGBoost')

  st.write("""
    my_xgb = XGBRegressor()
  """)


  data = student_info_assessments.dropna(subset=['score'])
  X = data.drop(columns=['id_student', 'score']).select_dtypes(include=[np.number])
  y = data['score']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  my_xgb = XGBRegressor()
  my_xgb.fit(X_train, y_train)

  xgb_fcst = my_xgb.predict(X_test)
  st.write("MSE ", mean_squared_error(y_test, xgb_fcst))
  st.write("MAE ", mean_absolute_error(y_test, xgb_fcst))
  st.write(r2_score(y_test, xgb_fcst))

  plot_xgb_fcst = xgb_fcst[1:100]
  plot_y_test = y_test[1:100]

  #Create a graph to compare the XGBoost forecast to the actuals
  plt.figure(figsize=(20,10))
  plt.plot(list(plot_xgb_fcst))
  plt.plot(list(plot_y_test))
  plt.legend(['actual', 'xgb forecast'])
  plt.ylabel('Score')
  plt.xlabel('Steps in test data')
  st.pyplot(plt.gcf())


def arima(df):

  st.subheader('ARIMA')

  st.write("""
  model = auto_arima(train_target,
                      start_p=1, start_q=1,
                      test='adf',       # Use ADF test to find optimal 'd'
                      max_p=3, max_q=3, # Maximum p and q
                      m=1,              # Frequency of series
                      d=0,           # Let model determine 'd'
                      seasonal=False,   # No seasonality
                      start_P=0,
                      trace=True,       # Print status
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True)    # Apply stepwise algorithm
  """)


  data = df[['date_submitted','score']].copy()

  # Set date_submitted as index]
  data = data.set_index('date_submitted')

  st.subheader('Testing for Stationarity')

  for col in list(data):
    d = nsdiffs(data[col],
            m=10,
            max_D=12,
            test='ch')

    st.write('Columns:: ', col, ' || d:: ', d)

  plot_acf(data['score'], lags=40)
  plot_pacf(data['score'], lags=40)
  st.pyplot(plt.gcf())

  data_daily = data.resample('W').mean()
  data_daily = data_daily.sort_index()
  data_daily = data_daily.interpolate(method='time')

  plt.plot(data_daily['score'])
  plt.title(f'Trend for score ')
  plt.xlabel('Date Submitted')
  plt.ylabel('Score')
  plt.show()

  adf_test = adfuller(data_daily['score'])
  # Output the results
  st.write('ADF Statistic: %f' % adf_test[0])
  st.write('p-value: %f' % adf_test[1])
  st.write('Since p-value < 0.05, stationarity doesnt exist (d = 0)')

  train_size = int(len(data_daily) * 0.8)
  train_data, test_data = data_daily.iloc[:train_size], data_daily.iloc[train_size:]

  # Target variable
  train_target = train_data['score']
  test_target = test_data['score']

  #Running Auto ARIMA
  model = auto_arima(train_target,
                        start_p=1, start_q=1,
                        test='adf',       # Use ADF test to find optimal 'd'
                        max_p=1, max_q=1, # Maximum p and q
                        m=1,              # Frequency of series
                        d=0,           # Let model determine 'd'
                        seasonal=False,   # No seasonality
                        start_P=0,
                        trace=True,       # Print status
                        error_action='ignore',
                        suppress_warnings=True,
                        stepwise=True)    # Apply stepwise algorithm

  st.subheader('Arima Model Summary')
  st.write(model.summary())

  st.subheader('Forecasting')

  #forecasting
  n_periods = len(test_target)
  forecast, conf_int = model.predict(n_periods=n_periods, return_conf_int=True)

  # Convert predictions to a DataFrame
  forecast_df = pd.DataFrame(forecast, index=test_target.index, columns=['Forecast'])
  conf_int_df = pd.DataFrame(conf_int, index=test_target.index, columns=['Lower CI', 'Upper CI'])

  mae = mean_absolute_error(test_target, forecast)
  rmse = np.sqrt(mean_squared_error(test_target, forecast))
  st.write(f'MAE: {mae}')
  st.write(f'RMSE: {rmse}')

  plt.figure(figsize=(12, 6))
  plt.plot(train_target, label='Train')
  plt.plot(test_target, label='Test')
  plt.plot(forecast_df, label='Forecast')
  plt.fill_between(conf_int_df.index,
                  conf_int_df['Lower CI'],
                  conf_int_df['Upper CI'],
                  color='k', alpha=.15)
  plt.legend()
  st.pyplot(plt.gcf())


# -------------------------------------Main Code--------------------------------------------
# -------------------------------------Main Code--------------------------------------------
# -------------------------------------Main Code--------------------------------------------
# -------------------------------------Main Code--------------------------------------------


# Title
st.title('Open University Learning Analytics Dataset (OULAD) Analysis')

# Subheading
st.markdown("### Dataset Visualization and Analysis")

# Sidebar for navigation
st.sidebar.header("Contents")
sections = [
    "Introduction", "Missing Value Analysis","Exploratory Data Analysis (EDA)", "Machine Learning (ML)"
]

start_date = pd.Timestamp('2000-01-01')

choice = st.sidebar.radio("Select a section:", sections)

# Introduction to Streamlit
if choice == "Introduction":

    st.header("Introduction to Dataset")
    st.markdown("""
    This dataset belongs to Open University Online Learning Platform (Also called as "Virtual Learning Environment(VLE)") that off-campus students use for accessing the course content, forum discussions, sending assessments and checking out assignment marks etc. It consists of 7 selected courses (mentioned as modules in the dataset). Different presentations indicated with letters "B" and "J" after year for semester 2 and semester 1 respectively.

    Additionally, the dataset includes student demographics such as location, age group, disability, education level, gender etc.
    Student assessment marks, interactions with the Virtual Learning Environment (VLE) are also included.

    It contains data about courses, students and their interactions with Virtual Learning Environment (VLE) for seven selected courses (called modules).
    Presentations of courses start in February and October - they are marked by “B” and “J” respectively. The dataset consists of tables connected using unique identifiers.
    """)

    st.title("Overview of the Datasets:")

    datasets = {
        "VLE": vle,
        "Student Interactions": vle_stu,
        "Student Registration": reg_stu,
        "Student Info": info_stu,
        "Student Assessments": as_stu,
        "Courses": course,
        "Assessments": assesment
    }

    dataset_choice = st.selectbox("Select Dataset for Overview:", list(datasets.keys()))

    if dataset_choice:
        st.subheader(f"Overview for {dataset_choice}")
        st.write(datasets[dataset_choice][:1000])

#--------------------------------------------------------------------------------------------------------------------------------

elif choice == "Missing Value Analysis":
    st.header("Missing Value Treatment")

    datasets = {
        "VLE": vle,
        "Student Interactions": vle_stu,
        "Student Registration": reg_stu,
        "Student Info": info_stu,
        "Student Assessments": as_stu,
        "Courses": course,
        "Assessments": assesment
    }

    dataset_choice = st.selectbox("Select Dataset for Missing Value Analysis:", list(datasets.keys()))

    if dataset_choice == 'VLE':
      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.subheader(f"Missing Value Treatment for {dataset_choice}")
      st.write("Since the 'Week From' and 'Week to' has around 82% missing values, it is best to drop these columns.")
      st.write ("vle.drop(columns=['week_from','week_to'],inplace=True)")

    elif dataset_choice == 'Student Interactions':
      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.write('No Missing Values')
      st.subheader('Transforming date to datetime')
      st.write("""
      - Define the start date
      start_date = pd.Timestamp('2000-01-01')

      - Convert 'date' to datetime
      vle_stu['date'] = vle_stu['date'].apply(lambda x: start_date + pd.Timedelta(days=x-1))
      """)

    elif dataset_choice == 'Student Registration':
      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.subheader(f"Missing Value Treatment for {dataset_choice}")
      st.write("Since the 'date_unregistration' has around 65% missing values, it is best to drop these columns.")

      st.subheader('Transforming date_registration to datetime')
      st.write("""
      Define the start date
      start_date = pd.Timestamp('2000-01-01')

      Convert 'date_submitted' to datetime
      reg_stu['date_registration'] = reg_stu['date_registration'].apply(lambda x: start_date + pd.Timedelta(days=x-1))
      """)

    elif dataset_choice == 'Student Info':
      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.subheader(f"Missing Value Treatment for {dataset_choice}")
      st.write("Since 'imd_band' has missing value it's better to impute it with dummy value")
      st.write("info_stu['imd_band'].fillna('unknown', inplace=True)  # Filling missing values with 'unknown'")

    elif dataset_choice == 'Student Assessments':
      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.subheader(f"Missing Value Treatment for {dataset_choice}")
      st.write("Score has missing values. Imputing the missing value with the mean of the score achieved that particular student")
      st.write("""
      student_means = as_stu.groupby('id_student')['score'].transform('mean')
      as_stu['score'].fillna(student_means, inplace=True)
      """)

      st.subheader('Transforming date_submitted to datetime')
      st.write("""
      Define the start date
      start_date = pd.Timestamp('2000-01-01')

      Convert 'date_submitted' to datetime
      as_stu['date_submitted'] = as_stu['date_submitted'].apply(lambda x: start_date + pd.Timedelta(days=x-1))
      """)

    elif dataset_choice == 'Courses':
      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.write('No Missing Values')

    elif dataset_choice == 'Assessments':

      st.subheader(f"Missing Value Analysis for {dataset_choice}")
      missingValueAssessment(datasets[dataset_choice])

      st.write("'Date' has around 5% missing values so it's better that we drop these missing values as we don't if this exam has been conducted or not")




#--------------------------------------------------------------------------------------------------------------------------------------

elif choice == "Exploratory Data Analysis (EDA)":

  start_date = pd.Timestamp('2000-01-01')

  datasets = [
        "VLE",
        "Student Interactions",
        "Student Registration",
        "Student Info",
        "Student Assessments",
        "Courses",
        "Assessments"
  ]

  dataset_choice = st.selectbox("Select Dataset for EDA:", datasets)

  if dataset_choice == 'VLE':
    st.subheader(f"EDA for {dataset_choice}")

    vle_cleaned = vle.copy()
    vle_cleaned.drop(columns=['week_from','week_to'],inplace=True)

    plotActivityCounts(vle_cleaned)

  elif dataset_choice == 'Student Interactions':

    vle_stu_cleaned = vle_stu.copy()
    # Convert 'date' to datetime
    vle_stu_cleaned['date'] = vle_stu_cleaned['date'].apply(lambda x: start_date + pd.Timedelta(days=x-1))


    student_vle_eda(vle_stu_cleaned)

  elif dataset_choice == 'Student Registration':

    reg_stu_cleaned = reg_stu.copy()
    reg_stu_cleaned.drop(columns=['date_unregistration'],inplace=True)
    reg_stu_cleaned['date_registration'] = pd.to_numeric(reg_stu_cleaned['date_registration'])
    reg_stu_cleaned = reg_stu_cleaned.dropna()
    reg_stu_cleaned['date_registration'] = reg_stu_cleaned['date_registration'].apply(lambda x: start_date + pd.Timedelta(days=x-1))

    stu_reg_eda(reg_stu_cleaned)

  elif dataset_choice == 'Student Info':
    info_stu_cleaned = info_stu.copy()
    info_stu_cleaned['imd_band'] = info_stu_cleaned['imd_band'].fillna('unknown', inplace=True)
    info_stu_cleaned['num_of_prev_attempts'] = info_stu_cleaned['num_of_prev_attempts'].fillna(info_stu['num_of_prev_attempts'].mean(), inplace=True)

    info_stu_eda(info_stu_cleaned)

  elif dataset_choice == 'Student Assessments':

    as_stu_cleaned = as_stu.copy()
    student_means = as_stu_cleaned.groupby('id_student')['score'].transform('mean')
    as_stu_cleaned['score'] = as_stu_cleaned['score'].fillna(student_means, inplace=True)
    as_stu_cleaned['date_submitted'] = as_stu_cleaned['date_submitted'].apply(lambda x: start_date + pd.Timedelta(days=x-1))

    as_stu_eda(as_stu_cleaned)

elif choice == "Machine Learning (ML)":

  student_info_assessments = data_merge(as_stu, info_stu)


  # Combine the assessments information
  student_info_assessments = data_merge(as_stu, info_stu)

  models = ['Introduction','Linear Regression', 'Decision Tree Regressor', 'XGBoost', 'ARIMA']

  model_choice = st.selectbox("Select Model for Prediction:", models)



  if model_choice == 'Introduction':
    st.subheader("Data Merging and Splitting for Modeling")

    st.write('Data Selected: Student Assessment & Student Information')
    st.write('data_merge(as_stu, info_stu)')
    st.write("Final Dataset Size:: 207319 rows × 15 columns")
    st.write ("Target Column: score")
    st.write("Date column:: date_submitted")

    st.write(student_info_assessments.head(150))

    st.write("Spliting the data into 80% training set and 20% test set")
    st.write("X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)")

  elif(model_choice == 'Linear Regression'):
    linear_reg(student_info_assessments)
  elif model_choice == 'Decision Tree Regressor':
    decision_tree(student_info_assessments)
  elif model_choice == 'XGBoost':
    xgboost_mdl(student_info_assessments)
  elif model_choice == 'ARIMA':
    arima(student_info_assessments)















Writing app.py


In [8]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.822s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [9]:
!streamlit run /content/app.py --server.port 8055 &>/content/logs.txt & curl ipv4.icanhazip.com

34.106.206.146


In [ ]:
!npx localtunnel --port 8055


npx: installed 22 in 1.539s
your url is: https://quick-lions-run.loca.lt
